# 第一阶段：ETT 数据预处理与探索性分析（EDA）

- 目标：
  - 熟悉 ETT 数据格式（时间序列、电力指标、油温 OT）。
  - 完成缺失值/异常值检测与清洗。
  - 完成特征缩放（标准化/归一化），导出清洗后数据。
  - 完成 OT 相关性与时序特性分析，可视化并导出图表。
- 数据源：`ETDataset-main/ETT-small/ETTh1.csv`, `ETTh2.csv`, `ETTm1.csv`, `ETTm2.csv`（可自行切换）。
- 运行环境：`Python (ETD-Env)`。
- 输出路径：`project/first/outputs/cleaned`（数据）、`project/first/outputs/figures`（图表）。


In [ ]:
# 基础设置与依赖导入
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from typing import Tuple, Dict

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf

# 路径设置
DATA_DIR = Path('ETDataset-main/ETT-small')
OUTPUT_DIR = Path('project/first/outputs')
FIG_DIR = OUTPUT_DIR / 'figures'
CLEAN_DIR = OUTPUT_DIR / 'cleaned'
FIG_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

# 可选数据文件
CSV_FILE = 'ETTh1.csv'  # 可改为 ETTh2.csv / ETTm1.csv / ETTm2.csv
DATA_PATH = DATA_DIR / CSV_FILE
print(f'使用数据文件: {DATA_PATH}')


In [ ]:
# 加载与基本检查
assert DATA_PATH.exists(), f'数据文件不存在: {DATA_PATH}'

df = pd.read_csv(DATA_PATH)
print('原始形状:', df.shape)
print('列名:', list(df.columns))

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)
df = df.drop_duplicates(subset=['date'])

# 简要预览
display(df.head())
display(df.describe(include='all'))

# 基本信息
missing_summary = df.isna().sum()
print('缺失值统计:\n', missing_summary)

dtypes_info = df.dtypes
print('数据类型:\n', dtypes_info)

# 可视化：OT 概览
plt.figure(figsize=(12, 3))
plt.plot(df['date'], df['OT'], linewidth=0.8)
plt.title('OT 概览')
plt.xlabel('date')
plt.ylabel('OT')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_ot_overview.png', dpi=150)
plt.show()


In [ ]:
# 缺失值与异常值检测
value_cols = [c for c in df.columns if c != 'date']

# 缺失值简单插补（记录缺失比例）
missing_ratio = df[value_cols].isna().mean()
print('缺失比例:\n', missing_ratio)

imputer = SimpleImputer(strategy='median')
df[value_cols] = imputer.fit_transform(df[value_cols])

# IQR 法检测异常值（标记但暂不删除）
Q1 = df[value_cols].quantile(0.25)
Q3 = df[value_cols].quantile(0.75)
IQR = Q3 - Q1

is_outlier = (df[value_cols] < (Q1 - 1.5 * IQR)) | (df[value_cols] > (Q3 + 1.5 * IQR))
outlier_rate = is_outlier.mean()
print('异常占比(列级):\n', outlier_rate)

df['outlier_any'] = is_outlier.any(axis=1)
print('含任一异常的样本比例:', df['outlier_any'].mean())

# 可视化：OT 的箱线图与分布
fig, axes = plt.subplots(1, 2, figsize=(10, 3))
sns.boxplot(y=df['OT'], ax=axes[0])
axes[0].set_title('OT 箱线图')
sns.histplot(df['OT'], kde=True, ax=axes[1], bins=50)
axes[1].set_title('OT 分布')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_ot_box_hist.png', dpi=150)
plt.show()


In [ ]:
# 清洗策略与缩放（可切换）
# 1) 对含异常的样本：可选择删除或 Winsorize（截尾）
REMOVE_OUTLIER_ROWS = False
WINSORIZE = True

clean_df = df.copy()

if REMOVE_OUTLIER_ROWS:
    clean_df = clean_df[~clean_df['outlier_any']].reset_index(drop=True)
    print('删除异常样本后形状:', clean_df.shape)

if WINSORIZE:
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    for col in value_cols:
        clean_df[col] = clean_df[col].clip(lower[col], upper[col])
    print('已对数值列进行截尾处理')

# 2) 特征缩放：选择标准化或归一化
USE_STANDARD_SCALER = True

scaler = StandardScaler() if USE_STANDARD_SCALER else MinMaxScaler()
scaled_values = scaler.fit_transform(clean_df[value_cols])
scaled_df = clean_df[['date']].copy()
scaled_df[value_cols] = scaled_values

print('缩放后预览:')
display(scaled_df.head())

# 导出清洗与缩放数据
clean_path = CLEAN_DIR / f'{CSV_FILE.replace(".csv", "")}_clean.csv'
scaled_path = CLEAN_DIR / f'{CSV_FILE.replace(".csv", "")}_scaled.csv'
clean_df.to_csv(clean_path, index=False)
scaled_df.to_csv(scaled_path, index=False)
print('已导出:', clean_path)
print('已导出:', scaled_path)


In [ ]:
# 与 OT 的相关性分析
corr = clean_df[value_cols].corr()
print('相关性矩阵（前 5 行）:')
display(corr.head())

# 与 OT 的 Pearson 相关系数
ot_corr = corr['OT'].sort_values(ascending=False)
print('与 OT 的相关性排序:')
print(ot_corr)

# 热力图
plt.figure(figsize=(8, 6))
sns.heatmap(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('特征相关性热力图')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_corr_heatmap.png', dpi=150)
plt.show()

# OT 与各负荷的散点关系
load_cols = [c for c in value_cols if c != 'OT']
num_to_plot = min(6, len(load_cols))
fig, axes = plt.subplots(2, (num_to_plot + 1) // 2, figsize=(12, 6))
axes = axes.flatten()
for i, col in enumerate(load_cols[:num_to_plot]):
    axes[i].scatter(clean_df[col], clean_df['OT'], s=2, alpha=0.3)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('OT')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_ot_scatter.png', dpi=150)
plt.show()


In [ ]:
# 时序特性：平稳性与自相关
ot_series = clean_df.set_index('date')['OT']

# ADF 检验
adf_stat, p_value, usedlag, nobs, crit, icbest = adfuller(ot_series.dropna(), autolag='AIC')
print(f'ADF Statistic: {adf_stat:.4f}, p-value: {p_value:.6f}')

# ACF/PACF
lag_n = 60 if 'm' not in CSV_FILE else 24 * 7  # 小时数据看 60, 分钟数据看一周
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sm.graphics.tsa.plot_acf(ot_series, lags=lag_n, ax=axes[0])
sm.graphics.tsa.plot_pacf(ot_series, lags=lag_n, ax=axes[1], method='ywm')
axes[0].set_title('OT ACF')
axes[1].set_title('OT PACF')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_ot_acf_pacf.png', dpi=150)
plt.show()

# 季节性可视化（小时/星期聚合）
df_hour = clean_df.copy()
df_hour['hour'] = df_hour['date'].dt.hour
hour_mean = df_hour.groupby('hour')['OT'].mean()

plt.figure(figsize=(8,3))
plt.plot(hour_mean.index, hour_mean.values)
plt.title('OT 按小时平均')
plt.xlabel('hour')
plt.ylabel('OT')
plt.tight_layout()
plt.savefig(FIG_DIR / f'{CSV_FILE}_ot_hourly_mean.png', dpi=150)
plt.show()


## 使用说明
- 在第二个代码单元中通过修改 `CSV_FILE` 切换数据源（ETTh1/ETTh2/ETTm1/ETTm2）。
- 通过 `REMOVE_OUTLIER_ROWS` 与 `WINSORIZE` 切换异常处理策略。
- 通过 `USE_STANDARD_SCALER` 切换 StandardScaler / MinMaxScaler。
- 运行全部单元后，清洗数据与缩放数据将分别导出到 `project/first/outputs/cleaned/`，图表导出到 `project/first/outputs/figures/`。
- 数据字段含义参考数据集官方说明与中文文档。

参考资料：数据集官方仓库 [`ETDataset`](https://github.com/zhouhaoyi/ETDataset)
